In [2]:
import logging

logger = logging.getLogger(__name__)

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from short_tracker.data import (
    query_all_sec_metadata, query_uk_si_disclosures, query_mkt_data, query_quotes,
    SHORT_URL_UK, DATE_COL, FUND_COL, ISIN_COL, SHORT_POS_COL,
    SHARE_ISSUER_COL, UK_DISCL_THRESHOLD, FCA_DATE_COL,
)
from short_tracker.processing import (
    check_cur_hist_discl_overlap, remove_dupl_shorts, ffill_discl_data, calc_fund_short_flow_bounds,
    extract_sec_tickers, process_mkt_data, subset_top_shorts,
)

NUM_SUBSET = 20
UK_MKT_TICKER = "VUKE"
ALPHAVANGATE_KEY = ""

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
discl_data, rept_date = query_uk_si_disclosures(SHORT_URL_UK) # exp_upd_time
rept_date

datetime.date(2022, 12, 30)

In [5]:
cur_discl = discl_data['current']

exp_max_discl_date = rept_date - timedelta(days=1)
max_discl_date = cur_discl[FCA_DATE_COL].max().date()

assert max_discl_date <= exp_max_discl_date
max_discl_date

datetime.date(2022, 12, 29)

In [6]:
# subset on current top N total overall shorts + top N individual shorts
top_sec_shorts, top_fund_shorts = subset_top_shorts(cur_discl, NUM_SUBSET)

isins = list({*top_fund_shorts[ISIN_COL].values, *top_sec_shorts[ISIN_COL].values})
print(len(isins))
isins[:5]

30


['GB0032089863',
 'GB00B132NW22',
 'GB0030927254',
 'GB00B4Y7R145',
 'GB0033195214']

In [7]:
sec_metadata, err_isins = query_all_sec_metadata(isins, {"idType": "ID_ISIN", "exchCode": "LN"})
len(err_isins)

0

In [8]:
isin_ticker_map = extract_sec_tickers(sec_metadata)
tickers = [UK_MKT_TICKER, *isin_ticker_map.values()]
query_tickers = [k.rstrip("/") + '.L' for k in tickers]

ticker_map = dict(zip(tickers, query_tickers))

tickers[:5]

['VUKE', 'NXT', 'ASHM', 'ASC', 'CURY']

In [60]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [ ]:
n_bdays_ago
rept_date

In [59]:
pd.Series(isin_ticker_map, name='ticker').rename_axis('isin').reset_index()

,isin,ticker
0,GB0032089863,NXT
1,GB00B132NW22,ASHM
2,GB0030927254,ASC
3,GB00B4Y7R145,CURY
4,GB0033195214,KGF
5,GB00B53P2009,JUP
6,GB00BYRJ5J14,PHP
7,GB00B019KW72,SBRY
8,GB00BJTNFH41,AO/
9,GB00BZ6STL67,MTRO


In [10]:
query_start = datetime.today().date() - timedelta(days=50)

mkt_data = {tkr: query_mkt_data(qry_tkr, query_start) for tkr, qry_tkr in ticker_map.items()}
quotes = {tkr: query_quotes(qry_tkr) for tkr, qry_tkr in ticker_map.items()}

In [11]:
SH_OUT_COL = 'shares_outstanding'

mkt_data_df = process_mkt_data(mkt_data)
quotes_df = pd.DataFrame(quotes)
sh_out = quotes_df.loc['sharesOutstanding'].rename(SH_OUT_COL)

missing_sh_out = list(sh_out[sh_out.isna()].index)

if missing_sh_out:
    logger.warning(f"No share outstanding data for tickers: {missing_sh_out}")

No share outstanding data for tickers: ['VUKE']


In [50]:
from short_tracker.utils import setup_logging, n_bdays_ago

In [53]:
sh_out_ = sh_out.rename_axis("ticker").rename('value').to_frame().reset_index()
sh_out_.loc[:, 'date'] = pd.to_datetime(n_bdays_ago(1))
sh_out_.loc[:, 'item'] = 'shout'

In [54]:
sh_out_

,ticker,value,date,item
0,VUKE,NaN,2022-12-30,shout
1,NXT,129263000,2022-12-30,shout
2,ASHM,712740992,2022-12-30,shout
3,ASC,100015000,2022-12-30,shout
4,CURY,1133490048,2022-12-30,shout
5,KGF,1939730048,2022-12-30,shout
6,JUP,546387008,2022-12-30,shout
7,PHP,1336489984,2022-12-30,shout
8,SBRY,2345060096,2022-12-30,shout
9,AO/,576899968,2022-12-30,shout


In [41]:
mkt_data_df.rename_axis(index=DATE_COL, columns='item').set_index("Ticker", append=True).stack().rename('value').reset_index()

,date,Ticker,item,value
0,2022-11-11,VUKE,Close,32.025002
1,2022-11-11,VUKE,Adj Close,32.025002
2,2022-11-11,VUKE,Volume,276196.000000
3,2022-11-14,VUKE,Close,32.305000
4,2022-11-14,VUKE,Adj Close,32.305000
...,...,...,...,...
3157,2022-12-29,VCP,Adj Close,494.000000
3158,2022-12-29,VCP,Volume,106818.000000
3159,2022-12-30,VCP,Close,481.000000
3160,2022-12-30,VCP,Adj Close,481.000000


datetime.date(2022, 12, 30)

In [15]:
mkt_data_df.assign(**{DATE_COL: rept_date})

,Close,Adj Close,Volume,Ticker,Position Date
2021-12-30,32.334999,32.334999,181089,VUKE,2022-12-30
2021-12-31,32.259998,32.259998,63784,VUKE,2022-12-30
2022-01-04,32.775002,32.775002,922919,VUKE,2022-12-30
2022-01-05,32.860001,32.860001,379070,VUKE,2022-12-30
2022-01-06,32.570000,32.570000,505078,VUKE,2022-12-30
...,...,...,...,...,...
2022-12-22,34.950001,34.950001,10006476,BOO,2022-12-30
2022-12-23,34.750000,34.750000,9713914,BOO,2022-12-30
2022-12-28,33.889999,33.889999,13647584,BOO,2022-12-30
2022-12-29,36.000000,36.000000,9983660,BOO,2022-12-30


In [13]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [ ]:
ALPHAVANTAGE_ENDPOINT = "https://www.alphavantage.co/query?"

import requests

from short_tracker.data import APIRateLimitException

# def query_mkt_data(ticker, api_key):
#     """Query Alphavantage for adjusted stock prices and """
#     def query_endpoint(fn):
#         av_args = {"function": fn, "symbol": ticker, "apikey": api_key}
#         resp = requests.get(ALPHAVANTAGE_ENDPOINT, params=av_args)
#         resp.raise_for_status()
#         data = resp.json()
        
#         if "Note" in data:
#             raise APIRateLimitException
#         elif "Error Message" in data:
#             raise ValueError(data["Error Message"])
#         return data
        
#     price = query_endpoint("TIME_SERIES_DAILY_ADJUSTED")
#     sec_info = query_endpoint("OVERVIEW")
#     return price, sec_info

# price, sec_info = query_mkt_data("RWI.L", ALPHAVANGATE_KEY)

In [18]:
# TODO: nice to have schemas defined like this but keeping things simple for now

from typing import List
from sqlalchemy import Column, Float, String, create_engine
# from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, constr

engine = create_engine("sqlite:///:memory:")

Base = declarative_base()


class DisclData(Base):
    __tablename__ = 'discl_data'
    fund = Column(String(200), primary_key=True, nullable=False)
    isin = Column(String(20), primary_key=True, nullable=False)
    share_issuer = Column(String(200), primary_key=True, nullable=False)
    short_position = Float()
    
    class Config:
        orm_mode = True


co_orm = DisclData(fund='a', isin='b', share_issuer='c', short_position=1.1231)
print(co_orm)

In [21]:
md = Base.metadata
md.create_all(engine)

In [22]:
md

FacadeDict({'discl_data': Table('discl_data', MetaData(), Column('fund', String(length=200), table=<discl_data>, primary_key=True, nullable=False), Column('isin', String(length=20), table=<discl_data>, primary_key=True, nullable=False), Column('share_issuer', String(length=200), table=<discl_data>, primary_key=True, nullable=False), schema=None)})

In [23]:
cur_discl

NameError: name 'cur_discl' is not defined

In [20]:
pd.read_sql_query('select * from discl_data', con=engine)

,fund,isin,share_issuer


In [33]:
cur_discl.to_sql(name='discl_data', con=engine, if_exists='append')

OperationalError: (sqlite3.OperationalError) table discl_data has no column named index
[SQL: INSERT INTO discl_data ("index", "Position Holder", "Name of Share Issuer", "ISIN", "Net Short Position (%)", "Position Date") VALUES (?, ?, ?, ?, ?, ?)]
[parameters: ((0, 'BlackRock Investment Management (UK) Limited', 'abrdn plc', 'GB00BF8Q6K64', 1.05, '2022-12-16 00:00:00.000000'), (1, 'Citadel Advisors Europe Limited', 'abrdn plc', 'GB00BF8Q6K64', 0.9, '2022-11-22 00:00:00.000000'), (2, 'Citadel Advisors LLC', 'abrdn plc', 'GB00BF8Q6K64', 1.0, '2022-12-29 00:00:00.000000'), (3, 'GLG Partners LP', 'abrdn plc', 'GB00BF8Q6K64', 1.15, '2022-12-16 00:00:00.000000'), (4, 'Qube Research & Technologies Limited', 'abrdn plc', 'GB00BF8Q6K64', 0.55, '2022-12-16 00:00:00.000000'), (5, 'Bronte Capital Management Pty Ltd', 'AFC ENERGY PLC', 'GB00B18S7B29', 0.5, '2022-03-31 00:00:00.000000'), (6, 'AKO Capital LLP', 'Alphawave IP Group PLC', 'GB00BNDRMJ14', 0.66, '2022-10-11 00:00:00.000000'), (7, 'KUVARI

In [34]:
cur_discl

,Position Holder,Name of Share Issuer,ISIN,Net Short Position (%),Position Date
0,BlackRock Investment Management (UK) Limited,abrdn plc,GB00BF8Q6K64,1.05,2022-12-16
1,Citadel Advisors Europe Limited,abrdn plc,GB00BF8Q6K64,0.90,2022-11-22
2,Citadel Advisors LLC,abrdn plc,GB00BF8Q6K64,1.00,2022-12-29
3,GLG Partners LP,abrdn plc,GB00BF8Q6K64,1.15,2022-12-16
4,Qube Research & Technologies Limited,abrdn plc,GB00BF8Q6K64,0.55,2022-12-16
...,...,...,...,...,...
295,GLG Partners LP,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.68,2022-11-11
296,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,GB00B5N0P849,0.81,2022-12-16
297,Marshall Wace LLP,WPP PLC,JE00B8KF9B49,0.79,2022-12-13
298,GLG Partners LP,XP Power Limited,SG9999003735,1.06,2022-11-10


In [127]:
from short_tracker.data_main import query_uk_si_disclosures_, TOP_N_SHORTS, query_ticker_map, query_mkt_data_, update_db, main

In [154]:
main()

No share outstanding data for tickers: ['VUKE']
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:128: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  
/home/nikul/Documents/Programming/short_tracker/src/short_tracker/data_main.py:129: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  


In [130]:
from sqlalchemy import create_engine

from short_tracker.config import (
    CONN_STR,
)

In [131]:
engine = create_engine(CONN_STR)

In [155]:
from short_tracker.config import (
    CONN_STR,
    MAX_DATA_AGE,
    TOP_N_SHORTS,
    UK_MKT_TICKER,
)
from short_tracker.schemas import (
    SEC_METADATA_TABLE,
    MKT_DATA_TABLE,
    SHORT_DISCL_TABLE,
)

isin_ticker_map = pd.read_sql_table(SEC_METADATA_TABLE, con=engine)
mkt_data = pd.read_sql_table(MKT_DATA_TABLE, con=engine, parse_dates=[DATE_COL])
discl_data = pd.read_sql_table(SHORT_DISCL_TABLE, con=engine, parse_dates=[DATE_COL])

In [140]:
isin_ticker_map

,Ticker,ISIN
0,ASC,GB0030927254
1,ITM,GB00B0130H42
2,WINE,GB00B021F836
3,KGF,GB0033195214
4,BOO,JE00BG6L7297
5,ABDN,GB00BF8Q6K64
6,HMSO,GB00BK7YQK64
7,TPK,GB00BK9RKT01
8,CURY,GB00B4Y7R145
9,SBRY,GB00B019KW72


In [156]:
mkt_data

,Ticker,item,date,value
0,VUKE,Close,2022-11-30,3.318000e+01
1,VUKE,Adj Close,2022-11-30,3.318000e+01
2,VUKE,Volume,2022-11-30,3.503490e+05
3,VUKE,Close,2022-12-01,3.311000e+01
4,VUKE,Adj Close,2022-12-01,3.311000e+01
...,...,...,...,...
2010,BRBY,sharesOutstanding,2022-12-31,3.796390e+08
2011,ASHM,sharesOutstanding,2022-12-31,7.127410e+08
2012,AO/,sharesOutstanding,2022-12-31,5.769000e+08
2013,JUP,sharesOutstanding,2022-12-31,5.463870e+08


In [244]:
latest_rpt_date = discl_data[DATE_COL].max()
lookback_date = n_bdays_ago(22, latest_rpt_date)
latest_rpt_date, latest_rpt_date

(Timestamp('2022-12-30 00:00:00'), Timestamp('2022-12-30 00:00:00'))

In [245]:
reindex_dates = pd.bdate_range(lookback_date, latest_rpt_date, name=DATE_COL)
reindex_dates

DatetimeIndex(['2022-11-30', '2022-12-01', '2022-12-02', '2022-12-05',
               '2022-12-06', '2022-12-07', '2022-12-08', '2022-12-09',
               '2022-12-12', '2022-12-13', '2022-12-14', '2022-12-15',
               '2022-12-16', '2022-12-19', '2022-12-20', '2022-12-21',
               '2022-12-22', '2022-12-23', '2022-12-26', '2022-12-27',
               '2022-12-28', '2022-12-29', '2022-12-30'],
              dtype='datetime64[ns]', name='date', freq='B')

In [246]:
def reindex_mkt_data(mkt_data, reindex_dates, ffill=True, bfill=False):
    """Reindex market data on the given dates with optional forward and backwards filling.
    
    Returns: a df of market data reindexed as above with the same columns as the input
    data (ticker, date, item, value).
    """
    mkt_data_ = mkt_data.pivot(index=DATE_COL, columns=TICKER_COL, values=[VALUE_COL]).reindex(reindex_dates)

    if ffill:
        mkt_data_ = mkt_data_.ffill()
    if bfill:
        mkt_data_ = mkt_data_.bfill()
    return mkt_data_.stack().reset_index()

def calc_ret(price : pd.Series) -> pd.Series:
    return price.set_index(DATE_COL)[[VALUE_COL]].pct_change()

In [247]:
sh_out = reindex_mkt_data(mkt_data[mkt_data[ITEM_COL]==SH_OUT_COL], reindex_dates, bfill=True)
sh_out = sh_out.rename(columns={VALUE_COL: SH_OUT_COL})
sh_out

,date,Ticker,sharesOutstanding
0,2022-11-30,ABDN,2.008410e+09
1,2022-11-30,AML,6.987570e+08
2,2022-11-30,AO/,5.769000e+08
3,2022-11-30,ASC,1.000150e+08
4,2022-11-30,ASHM,7.127410e+08
...,...,...,...
685,2022-12-30,RWI,8.022000e+07
686,2022-12-30,SBRY,2.345060e+09
687,2022-12-30,TPK,2.125090e+08
688,2022-12-30,VCP,1.150010e+08


In [252]:
RET_COL = "Return"

adj_close = reindex_mkt_data(mkt_data[mkt_data[ITEM_COL]=='Adj Close'], reindex_dates, bfill=True)
adj_close = adj_close.sort_values(by=[TICKER_COL, DATE_COL])
returns = adj_close.groupby(TICKER_COL).apply(calc_ret).rename(columns={VALUE_COL: RET_COL}).reset_index()
returns

,Ticker,date,Return
0,ABDN,2022-11-30,NaN
1,ABDN,2022-12-01,0.030864
2,ABDN,2022-12-02,0.006487
3,ABDN,2022-12-05,-0.013386
4,ABDN,2022-12-06,0.015578
...,...,...,...
708,WINE,2022-12-26,0.000000
709,WINE,2022-12-27,0.000000
710,WINE,2022-12-28,0.013589
711,WINE,2022-12-29,0.017350


In [265]:
BM_RET_COL = "Benchmark Return"
bm_returns = returns[returns[TICKER_COL]==UK_MKT_TICKER].rename(columns={RET_COL: BM_RET_COL}).drop(columns=[TICKER_COL])
bm_returns

,date,Benchmark Return
667,2022-11-30,NaN
668,2022-12-01,-0.002110
669,2022-12-02,-0.000453
670,2022-12-05,0.003928
671,2022-12-06,-0.006321
672,2022-12-07,-0.004240
673,2022-12-08,-0.002738
674,2022-12-09,-0.000076
675,2022-12-12,-0.004042
676,2022-12-13,0.008423


In [267]:
def join_analysis_data(discl_data, isin_ticker_map, sh_out, returns, bm_returns):
    """Join a df of disclosures data with a df of tickers + isins and
    market data (sh out, returns) dfs on date + ticker"""
    discl_data_ = discl_data.merge(isin_ticker_map, on=ISIN_COL, how='left')
    
    for df in [sh_out, returns]:
        discl_data_ = discl_data_.merge(df, on=[DATE_COL, TICKER_COL], how='left')
        
    return discl_data_.merge(bm_returns, on=[DATE_COL], how='left')

discl_data_ = join_analysis_data(discl_data, isin_ticker_map, sh_out, returns, bm_returns)
discl_data_

,ISIN,Position Holder,Name of Share Issuer,date,Net Short Position (%),Ticker,sharesOutstanding,Return,Benchmark Return
0,GB00BF8Q6K64,BlackRock Investment Management (UK) Limited,abrdn plc,2022-12-30,1.05,ABDN,2.008410e+09,-0.002109,-0.005182
1,GB00BF8Q6K64,Citadel Advisors Europe Limited,abrdn plc,2022-12-30,0.90,ABDN,2.008410e+09,-0.002109,-0.005182
2,GB00BF8Q6K64,Citadel Advisors LLC,abrdn plc,2022-12-30,1.00,ABDN,2.008410e+09,-0.002109,-0.005182
3,GB00BF8Q6K64,GLG Partners LP,abrdn plc,2022-12-30,1.15,ABDN,2.008410e+09,-0.002109,-0.005182
4,GB00BF8Q6K64,Qube Research & Technologies Limited,abrdn plc,2022-12-30,0.55,ABDN,2.008410e+09,-0.002109,-0.005182
...,...,...,...,...,...,...,...,...,...
295,GB00B5N0P849,GLG Partners LP,WOOD GROUP (JOHN) PLC,2022-12-30,0.68,NaN,NaN,NaN,-0.005182
296,GB00B5N0P849,Qube Research & Technologies Limited,WOOD GROUP (JOHN) PLC,2022-12-30,0.81,NaN,NaN,NaN,-0.005182
297,JE00B8KF9B49,Marshall Wace LLP,WPP PLC,2022-12-30,0.79,NaN,NaN,NaN,-0.005182
298,SG9999003735,GLG Partners LP,XP Power Limited,2022-12-30,1.06,NaN,NaN,NaN,-0.005182


In [268]:
REL_RET_COL = "Relative Return"

discl_data_.loc[:, REL_RET_COL] = discl_data_[RET_COL] - discl_data_[BM_RET_COL]

In [272]:
AMOUNT_COL = "Amount"

discl_data_.loc[:, AMOUNT_COL] = discl_data_[SHORT_POS_COL] * discl_data_[SH_OUT_COL]

discl_data_[AMOUNT_COL] * discl_data_[RET_COL]

0     -4.447769e+06
1     -3.812373e+06
2     -4.235970e+06
3     -4.871366e+06
4     -2.329784e+06
           ...     
295             NaN
296             NaN
297             NaN
298             NaN
299             NaN
Length: 300, dtype: float64

In [251]:
discl_data_[discl_data_[TICKER_COL]=='ASC']

,ISIN,Position Holder,Name of Share Issuer,date,Net Short Position (%),Ticker,sharesOutstanding,Return
16,GB0030927254,AHL Partners LLP,ASOS PLC,2022-12-30,0.90,ASC,100015000.0,-0.022967
17,GB0030927254,Citadel Advisors LLC,ASOS PLC,2022-12-30,0.81,ASC,100015000.0,-0.022967
18,GB0030927254,GAM International Management Limited,ASOS PLC,2022-12-30,1.78,ASC,100015000.0,-0.022967
19,GB0030927254,Marshall Wace LLP,ASOS PLC,2022-12-30,3.02,ASC,100015000.0,-0.022967
20,GB0030927254,Squarepoint Ops LLC,ASOS PLC,2022-12-30,0.88,ASC,100015000.0,-0.022967
